<a href="https://colab.research.google.com/github/joehawkens/MachineLearning/blob/main/MODULE_3_Thursday.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**MODULE 3**

# **Module Resources**

**Problem:** Regression <n>

**Target:** Home Price

- Module Overview: https://byui-cse.github.io/cse450-course/module-03/
- Data Dictionary: https://byui-cse.github.io/cse450-course/module-03/housing-dictionary.txt
- Dataset: https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/housing.csv
- Holdout Dataset: https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/housing_holdout_test.csv
- Holdout Mini Dataset: https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/housing_holdout_test_mini.csv
- Module Hints: https://byui-cse.github.io/cse450-course/module-03/hints.html


# **Data Normalization and Cleaning**
- Are there outliers that will skew the data?
- Is there any misssing data?

In [2]:
import pandas as pd
import altair as alt
house_data = pd.read_csv("https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/housing.csv")

# Finding missing values
# Check if the DataFrame contains any missing data:
if house_data.isnull().any().any():
    print('The DataFrame contains missing data')
else:
    print('The DataFrame does not contain missing data')


# Features I think won't be useful:

# lat - Latitude - We already havea  zip code.
# long - Longitude - We already have a zip code.
# id - Unique ID for each home sold - This is a database key, it has nothing to do with the home price.

# All the features I think have potential:

# date - Date of the home sale
# price - Price of each home sold
# bedrooms - Number of bedrooms
# bathrooms - Number of bathrooms, where .5 accounts for a room with a toilet but no shower
# sqft_living - Square footage of the apartments interior living space
# sqft_lot - Square footage of the land space
# floors - Number of floors
# waterfront - A dummy variable for whether the apartment was overlooking the waterfront or not
# view - An index from 0 to 4 of how good the view of the property was
# condition - An index from 1 to 5 on the condition of the apartment,
# grade - An index from 1 to 13, where 1-3 falls short of building construction and design, 7 has an average level of construction and design, and 11-13 have a high quality level of construction and design.
# sqft_above - The square footage of the interior housing space that is above ground level
# sqft_basement - The square footage of the interior housing space that is below ground level
# yr_built - The year the house was initially built
# yr_renovated - The year of the house’s last renovation
# zipcode - What zipcode area the house was listed in
# sqft_living15 - The square footage of interior housing living space for the nearest 15 neighbors
# sqft_lot15 - The square footage of the land lots of the nearest 15 neighbors



house_data['price'].describe()

The DataFrame does not contain missing data


count    2.000000e+04
mean     5.394367e+05
std      3.664334e+05
min      7.500000e+04
25%      3.220000e+05
50%      4.500000e+05
75%      6.416250e+05
max      7.700000e+06
Name: price, dtype: float64

# **Data Exploration**

## **PRICE**


In [37]:
house_data['price'].value_counts()

price_scaled = house_data['price'] / 10000

ranges = list(range(0, 6000000, 250000))  # Create ranges from 0 to 10 million with 250,000 intervals
labels = [f'{r/1000}K-{(r+250000)/1000}K' for r in ranges[:-1]]  # Create labels for the ranges

house_data['price_range'] = pd.cut(house_data['price'], ranges, labels=labels)

count_data = house_data['price_range'].value_counts().reset_index()
count_data.columns = ['Price Range', 'Count']

chart = alt.Chart(count_data).mark_bar().encode(
    x=alt.X('Price Range:O', title='Price Range'),
    y=alt.Y('Count:Q', title='Count'),
).properties(
    title='Count of House Prices by Range'
).configure_axis(
    labelFontSize=12,
    titleFontSize=14
).configure_title(
    fontSize=16
)

chart

max = max(house_data['price'])
print(max)


7700000.0


## **SQFT_BASEMENT**

In [4]:
house_data['sqft_basement'].value_counts()



sampled_data = house_data.sample(5000)  # Create a random sample of 5000 rows

chart = alt.Chart(sampled_data).mark_bar().encode(
    x='sqft_basement:O',
    y='count()'
).properties(
    title='Value Counts of sqft_basement'
)

chart


alt.Chart(...)

## **BEDROOMS**

In [5]:
# Group the data by the number of bedrooms and calculate the average price for each category
price_avg = house_data.groupby('bedrooms')['price'].mean().reset_index()

# Create an Altair bar chart
chart = alt.Chart(price_avg).mark_bar().encode(
    x=alt.X('bedrooms:O', axis=alt.Axis(title='Number of Bedrooms')),
    y=alt.Y('price:Q', axis=alt.Axis(title='Price')),
    tooltip=['bedrooms:O', 'price:Q']
).properties(
    title='Average Price by Number of Bedrooms'
)

# Display the chart
chart


# nine_bedroom = house_data[house_data['bedrooms'] == 9]
# nine_bedroom['price'].value_counts()
house_data['bedrooms'].value_counts()

bedrooms_filtered = house_data[(house_data['bedrooms'] >= 0) & (house_data['bedrooms'] <= 5)]

correlation = bedrooms_filtered['bedrooms'].corr(bedrooms_filtered['price'])
correlation

0.31394428711439604

## **YR_RENOVATED**

In [6]:
house_data['yr_renovated'].value_counts()

0       19171
2014       86
2005       34
2003       34
2013       33
        ...  
1934        1
1951        1
1948        1
1954        1
1956        1
Name: yr_renovated, Length: 70, dtype: int64

## **DATE BUILT**

In [28]:
import numpy as np
from IPython.display import display

# Step 1: Calculate the decade for each year
house_data['decade_built'] = (house_data['yr_built'] // 10) * 10

# Step 2: Create bins for the decades
decade_bins = np.arange(house_data['decade_built'].min(), house_data['decade_built'].max() + 10, 10)

# Step 3: Assign each year to its corresponding decade bin
house_data['decade_built'] = pd.cut(house_data['yr_built'], bins=decade_bins, labels=decade_bins[:-1])

# Print the resulting DataFrame
house_data['decade_built'].value_counts()

# Assuming you have a DataFrame named "house_data" with columns "decade_built" and "housing_price"

# Enable Altair rendering in Google Colab
# alt.renderers.enable('colab')

# Group the data by decade and calculate the average housing price for each decade
decade_price_avg = house_data.groupby('decade_built')['price'].mean().reset_index()

# Create an Altair bar chart
chart = alt.Chart(decade_price_avg).mark_bar().encode(
    x='decade_built:O',
    y='price:Q',
    tooltip=['decade_built:O', 'price:Q']
).properties(
    title='Average Housing Price by Decade Built'
)

# Display the chart
display(chart)
house_data.head(5)

alt.Chart(...)

,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,...,zipcode,lat,long,sqft_living15,sqft_lot15,price,price_range,decade_built,sqft_lot_bin,month
0,1565930130,2014-11-04,4,3.25,3760,4675,2.0,0,0,3,...,98038,47.3862,-122.048,3280,4033,429900.0,250.0K-500.0K,2000,"(0, 30000]",11
1,3279000420,2015-01-15,3,1.75,1460,7800,1.0,0,0,2,...,98023,47.3035,-122.382,1310,7865,233000.0,0.0K-250.0K,1970,"(0, 30000]",1
2,194000575,2014-10-14,4,1.00,1340,5800,1.5,0,2,3,...,98116,47.5658,-122.389,1900,5800,455000.0,250.0K-500.0K,1910,"(0, 30000]",10
3,2115510160,2014-12-08,3,1.75,1440,8050,1.0,0,0,3,...,98023,47.3187,-122.390,1790,7488,258950.0,250.0K-500.0K,1980,"(0, 30000]",12
4,7522500005,2014-08-15,2,1.50,1780,4750,1.0,0,0,4,...,98117,47.6859,-122.395,1690,5962,555000.0,500.0K-750.0K,1940,"(0, 30000]",8


## **FLOORS**

In [8]:
house_data['floors'].value_counts()


# Group the data by the number of floors and calculate the average home price for each category
floor_price_avg = house_data.groupby('floors')['price'].mean().reset_index()

# Create an Altair bar chart
chart = alt.Chart(floor_price_avg).mark_bar().encode(
    x='floors:O',
    y='price:Q',
    tooltip=['floors:O', 'price:Q']
).properties(
    title='Average Home Price by Floors'
)

# Display the chart
chart


alt.Chart(...)

## **CONDITION** - Use scale

In [30]:
house_data['condition'].value_counts()

# Assuming you have a DataFrame named "house_data" with columns "condition" and "home_price"

# Group the data by the condition and calculate the average home price for each category
condition_price_avg = house_data.groupby('condition')['price'].mean().reset_index()

# Create an Altair bar chart
chart = alt.Chart(condition_price_avg).mark_bar().encode(
    x='condition:O',
    y='price:Q',
    tooltip=['condition:O', 'price:Q']
).properties(
    title='Average Home Price by Condition'
)

# Display the chart
chart

#house_data['condition'].value_counts()




alt.Chart(...)

## **VIEW**

In [10]:
house_data['view'].value_counts()

# Assuming you have a DataFrame named "house_data" with columns "condition" and "home_price"

# Group the data by the condition and calculate the average home price for each category
condition_price_avg = house_data.groupby('view')['price'].mean().reset_index()

# Create an Altair bar chart
chart = alt.Chart(condition_price_avg).mark_bar().encode(
    x='view:O',
    y='price:Q',
    tooltip=['view:O', 'price:Q']
).properties(
    title='Average Home Price by View'
)

# Display the chart
chart


house_data['view'].value_counts()



0    18030
2      889
3      471
1      311
4      299
Name: view, dtype: int64

## **SQFT_LOT**

In [11]:
house_data['sqft_lot'].value_counts()

5000     337
6000     258
4000     235
7200     206
7500     112
        ... 
17817      1
9001       1
12855      1
49928      1
6914       1
Name: sqft_lot, Length: 9297, dtype: int64

In [12]:
# Define the binning parameters
bin_width = 30000  # Adjust bin width as per your preference

# Create a binned column for sqft_lot and convert to string representation
house_data['sqft_lot_bin'] = pd.cut(house_data['sqft_lot'], bins=range(0, int(house_data['sqft_lot'].max()) + bin_width, bin_width)).astype(str)

# Group the data by the sqft_lot bin and calculate the average price for each bin
price_avg = house_data.groupby('sqft_lot_bin')['price'].mean().reset_index()

# Create an Altair bar chart
chart = alt.Chart(price_avg).mark_bar().encode(
    x=alt.X('sqft_lot_bin:O', axis=alt.Axis(title='Lot Size')),
    y=alt.Y('price:Q', axis=alt.Axis(title='Price')),
    tooltip=['sqft_lot_bin:O', 'price:Q']
).properties(
    title='Average Price by Lot Size'
)

# Display the chart
chart

alt.Chart(...)

## **WATERFRONT**

In [13]:
house_data['waterfront'].value_counts()

# Assuming you have a DataFrame named "house_data" with columns "condition" and "home_price"

# Group the data by the condition and calculate the average home price for each category
condition_price_avg = house_data.groupby('waterfront')['price'].mean().reset_index()

# Create an Altair bar chart
chart = alt.Chart(condition_price_avg).mark_bar().encode(
    x='waterfront:O',
    y='price:Q',
    tooltip=['waterfront:O', 'price:Q']
).properties(
    title='Average Home Price by Waterfront Property'
)

# Display the chart
chart


house_data['waterfront'].value_counts()



0    19849
1      151
Name: waterfront, dtype: int64

## **DATE**

In [14]:
import datetime
house_data['date'].value_counts()
month_numbers = []

# for date_string in house_data['date']:
#     try:
#         date_object = datetime.datetime.strptime(date_string, "%Y%m%dT%H%M%S")
#         month_number = date_object.month
#         month_numbers.append(month_number)
#     except ValueError:
#         print(f"Ignoring invalid date string: {date_string}")


df = pd.DataFrame({'Month': month_numbers})

df_agg = df.groupby('Month').size().reset_index(name='Count')

chart = alt.Chart(df_agg).mark_bar().encode(
    alt.X('Month:O', title='Month'),
    alt.Y('Count:Q', title='Count')
).properties(
    title='Count of Events by Month'
)

chart

# Housing price by month sold. ======================================================================

house_data['date'] = pd.to_datetime(house_data['date'])

# Extract month from 'date' column
house_data['month'] = house_data['date'].dt.month

# Calculate the average sell price for each month
average_price_by_month = house_data.groupby('month')['price'].mean().reset_index()

# Create a bar chart using Altair
chart = alt.Chart(average_price_by_month).mark_bar().encode(
    alt.X('month:O', title='Month', sort=alt.EncodingSortField(field='price', op='mean', order='descending')),
    alt.Y('price:Q', title='Average Sell Price')
).properties(
    title='Average Sell Price by Month'
)

# Display the chart
chart


# Housing price by year sold ======================================================================
# house_data['date'] = pd.to_datetime(house_data['date'])

# house_data['year'] = house_data['date'].dt.year

# average_price_by_year = house_data.groupby('year')['price'].mean().reset_index()

# Create a bar chart using Altair
# chart = alt.Chart(average_price_by_year).mark_bar().encode(
#     alt.X('year:O', title='Year'),
#     alt.Y('price:Q', title='Average Sell Price')
# ).properties(
#     title='Average Sell Price by Year'
# )

#chart
# house_data['year'] = house_data['date'].dt.year
# house_data['year'].value_counts()


alt.Chart(...)

## **CORRELATION**

In [15]:
# Calculate the correlation between each feature and price
feature_data = house_data[['date', 'price', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade', 'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode', 'sqft_living15', 'sqft_lot15']]
correlation_with_price = feature_data.corr()['price'].drop('price')
correlation_df = correlation_with_price.reset_index().rename(columns={'index': 'feature', 'price': 'correlation'})

# Creae heatmap in altair:
heatmap = alt.Chart(correlation_df).mark_rect().encode(
    x='feature:O',
    y=alt.Y('correlation:O', axis=alt.Axis(format='0.2f')),
    color='correlation:Q'
).properties(
    width=400,
    height=300,
    title='Correlation with Price Heatmap'
)


# 1 = high positive correlation, 0 = no correlation, -1 = high negative correlation
heatmap
correlation_with_price.sort_values()

<ipython-input-15-112aac17f914>:3: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  correlation_with_price = feature_data.corr()['price'].drop('price')


zipcode         -0.055857
condition        0.035146
yr_built         0.054187
sqft_lot15       0.082413
sqft_lot         0.090317
yr_renovated     0.121534
floors           0.254428
waterfront       0.260133
bedrooms         0.307281
sqft_basement    0.325183
view             0.395961
bathrooms        0.527989
sqft_living15    0.585374
sqft_above       0.607082
grade            0.666447
sqft_living      0.703920
Name: price, dtype: float64

In [16]:
feature_data = house_data[['date', 'price', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade', 'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode', 'sqft_living15', 'sqft_lot15']]

# Calculate the correlation matrix
correlation_matrix = feature_data.corr()

# Reset the index to convert the correlation matrix into a dataframe
correlation_df = correlation_matrix.reset_index()

# Melt the dataframe to convert it into long format for heatmap visualization
melted_df = pd.melt(correlation_df, id_vars='index', value_vars=correlation_df.columns[1:], var_name='feature1', value_name='correlation')

# Create the heatmap using Altair
heatmap = alt.Chart(melted_df).mark_rect().encode(
    x='index:O',
    y='feature1:O',
    color='correlation:Q'
).properties(
    width=300,
    height=300,
    title='Correlation Heatmap'
)

# Display the heatmap
heatmap

<ipython-input-16-315eb6242098>:4: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  correlation_matrix = feature_data.corr()


alt.Chart(...)

# **Feature Selection**
- We need a metric to determine which features are most useful in determining home price.


### High Positive Correlation with Price:
- Sqft_living   = 0.70
- Grade         = 0.66
- sqft_above    = 0.60
- sqft_living15 = 0.58
- bathrooms     = 0.52

### High Negative Correlation with Price:
- None (yet discovered)

### To be determined:
- Condition
- Zip Code (High income zip codes sell for more)
- Year Built (I binned the homes into decades they were built, there's some unique distributions worth looking into)
- Date (May, April, July, June are when the most homes are sold)


### **Ignore** these features:
- sqft_lot15                (almost no correlation)
- sqft_lot                  (almost no correlation)
- Lat                       (Useless data point)
- Long                      (Useless data point)
- Id                        (Only used in the database to store the row)
- yr_renovated              (Over 95% of the data falls under 0 - no data)
- bedrooms                  (very low positive correlation + outliers skew data)
- view                      (Highly imbalanced distribution among values, over 95% is 0)
- waterfront                (Highly imbalanced distribution among values, over 99% is 0)
- sqft_basement             (Over 95% of the data falls under 0 - no data)

## **FEATURE CLEANING**

### Price

In [17]:
house_data['price'] # Probably needs to be scaled, too widely distributed.

0        429900.0
1        233000.0
2        455000.0
3        258950.0
4        555000.0
           ...   
19995    378000.0
19996    399950.0
19997    575000.0
19998    245000.0
19999    315000.0
Name: price, Length: 20000, dtype: float64

### **Sqft_living**

In [18]:
house_data['sqft_living']

0        3760
1        1460
2        1340
3        1440
4        1780
         ... 
19995    1000
19996    3087
19997    2120
19998     380
19999    3130
Name: sqft_living, Length: 20000, dtype: int64

# **XGBoost Model**

TEST AND TRAIN SETS

In [19]:
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split

X = house_data[['sqft_living', 'grade', 'sqft_above', 'sqft_living15', 'bathrooms']]
y = house_data['price']


# How do we want to do the testing size?
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

TRAINING

In [32]:
model = XGBRegressor()
             
model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

PREDICTIONS

In [33]:
predictions = model.predict(X_test)
predictions

array([657998.  , 342029.28, 645846.2 , ..., 392519.94, 688018.75,
       422863.6 ], dtype=float32)

RESULTS

In [38]:
from sklearn.metrics import mean_squared_error
# holdout_mini = pd.read_csv("https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/housing_holdout_test_mini.csv")


result = mean_squared_error(y_test, predictions, squared=False)
result # Not good.

244330.90211018175